In [ ]:
import sharp
import h1data
import bq

import kaleido

import os

import matplotlib.pyplot as plt
import numpy as np
import spectral.io.envi as envi

In [ ]:
datapath = os.path.join(os.getcwd(),"data")

In [ ]:
h = h1data.h1data("data/20221107_CaptureDL_sudan_tl_2022_11_04T08_31_09")

In [ ]:
h.show_l1a_cube()

In [ ]:
h.show_raw_cube()

In [ ]:
s_list = []
brisque_scores = []
titles = []
for dataset in os.listdir(datapath):
    full_path = os.path.join(datapath,dataset)
    
    if not "CaptureDL" in full_path:
        continue
     
    hdata = h1data.h1data(full_path)
    sdata = sharp.sharp.fromh1data(hdata)
    s_list.append(sdata)
    
    sdata.evaluate()
    
    brisque_score = sdata.brisque["initial"]    
    brisque_scores.append(brisque_score)
    
    title = str()
    title = title.join([full_path.split("/")[-1].split("_")[2], " ", full_path.split("/")[-1].split("_")[0]])
    print(title)
    titles.append(title)
    
brisque_scores = np.array(brisque_scores)
    

In [ ]:
import importlib
importlib.reload(bq)

#brisque_scores = []
#for ind, sobj in enumerate(sdata):
#    print(f"Scene: {titles[ind]}, Progress: {ind}/{len(sdata)}")
#    sobj.evaluate()

brisque_scores = np.array(brisque_scores)

#bq.plotScoreStatistics(hdata.spec_coefficients[10:], brisque_scores[:,10:] ,0.01)
bq.plotScoreStatistics(hdata.spec_coefficients[10:], brisque_scores[[0,1,2,4,5,6,7,8],10:] ,0.02)
bq.plotAllScores(hdata.spec_coefficients[10:], brisque_scores[:,10:] , titles)

In [ ]:
import importlib
importlib.reload(sharp)
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

fig = make_subplots(rows=len(s_list), cols=1,
                   subplot_titles = titles)

for i, sharp_obj in enumerate(s_list):
    figure = px.imshow(np.rot90(sharp_obj.initial_cube[:,:,sharp_obj.sharpest_band_index]))
    for trace in range(len(figure["data"])):
        fig.append_trace(figure["data"][trace], row=1+i, col=1)

fig.update_layout(
    autosize=False,
    minreducedheight=i*200,
    height=i*250,
)
    
fig.show()
#sharp.plot_sharp_list(s_list)
#fig.write_image("fig/all.pdf")

In [ ]:
from numpy.linalg import svd

image = sdata.initial_cube
sh_ind = 47

img_variable_form = np.reshape(image, (image.shape[0] * image.shape[1], image.shape[2]))
U, S, Vh = svd(img_variable_form, full_matrices=False)
principal_components = np.dot(U, np.diag(S))

component_cube = np.reshape(principal_components, (image.shape[0], image.shape[1], image.shape[2]))


In [ ]:
from skimage.exposure import match_histograms

matched_sharpest_band = match_histograms(image[:,:,47], component_cube[:,:,0])

fig = px.imshow(matched_sharpest_band - component_cube[:,:,0])
fig.show()



In [ ]:
fixed_component_cube = component_cube
fixed_component_cube[:,:,0] = matched_sharpest_band
fixed_cc_variable_form = np.reshape(fixed_component_cube, (image.shape[0] * image.shape[1], image.shape[2]))

sharpend_variable_form = np.dot(fixed_cc_variable_form, Vh)
sharpend_cube = np.reshape(sharpend_variable_form, (image.shape[0], image.shape[1], image.shape[2]))

In [ ]:
sharpend_bq = bq.scoreCube(sharpend_cube)

In [ ]:
fig = go.Figure([
    go.Scatter(
    name="sharpend",
    x=hdata.spec_coefficients[10:],
    y=sharpend_bq[10:],
    mode="lines"
    ),
    go.Scatter(
    name="original",
    x=hdata.spec_coefficients[10:],
    y=brisque_score[10:],
    mode="lines"
    ),
])

fig.show()
fig.write_html("fig/initial_result.html")


In [ ]:
bq_scores_original = []
bq_scores_sharpend = []
for ind, sobj in enumerate(s_list):
    print(f"Scene: {titles[ind]}, Progress: {ind}/{len(s_list)}")
    sobj.sharpen()
    sobj.evaluate()
    
    bq_original = sobj.brisque["initial"]    
    bq_scores_original.append(bq_original)
    
    bq_sharpend = sobj.brisque["sharpend"]    
    bq_scores_sharpend.append(bq_sharpend)
        
bq_scores_original = np.array(bq_scores_original)
bq_scores_sharpend = np.array(bq_scores_sharpend)

In [ ]:
fig = go.Figure([
    go.scatter(
    
    )
])